# This is the code we are using to re-create chase's tool for the infauna data 

First we import the necessary packages

In [165]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

Then we read in the raw data and prepare it. the 'initial' and 'qc' dataframes will be used to build the report. The 'original_report' is just the original discrepancy report which will later be compared with the one generated with python.

In [166]:
initial = pd.read_excel("data/Infauna_QC_B13_OCSD.xlsm", sheet_name = 'Original_Data')
qc = pd.read_excel("data/Infauna_QC_B13_OCSD.xlsm", sheet_name = 'QC_Data')
qc.rename(columns={" SITE": "SITE"}, inplace=True)
original_report = pd.read_excel("data/Infauna_QC_B13_OCSD.xlsm", sheet_name = 'Discrepancy_Report')

Here we build the dataframe that will become the discrepancy report by merging the initial and QC dataframes

In [167]:
python_discrepancy_report = pd.merge(initial, qc, left_on = ['SITE', 'ORIGINAL SPECIES'], right_on = ['SITE', 'QC SPECIES'], how = 'outer')
python_discrepancy_report = python_discrepancy_report[['SITE', 'ORIGINAL SPECIES', 'ORIGINAL ABUNDANCE', 'ORIGINAL VOUCHER', 'QC SPECIES', 'QC ABUNDANCE']]

Now we define out function that will be applied row by row to the 'output' dataframe. This function will build the 'Match/Not Match' and 'Type' columns. It is built based on the VBA code that I saw in the Infauna_QC_B13_OCSD.xlsm workbook.

In [168]:
def discrepancy_check(row):
    'A function to be applied to the "output" dataframe row by row'
    if ((pd.isnull(row['ORIGINAL SPECIES'])) or (row['ORIGINAL SPECIES'] == '')):
        row['Match/Not Match'] = 'Not Match'
        row['Type'] = 'ID'
    else:
        if ((pd.isnull(row['QC SPECIES'])) or (row['QC SPECIES'] == '')):
            if row['ORIGINAL ABUNDANCE'] == row['ORIGINAL VOUCHER']:
                row['Match/Not Match'] = 'Match'
                row['Type'] = ''
            else:
                row['Match/Not Match'] = 'Not Match'
                row['Type'] = 'ID'
        else:
            if not pd.isnull(row['ORIGINAL VOUCHER']):
                if row['QC ABUNDANCE'] == row['ORIGINAL ABUNDANCE'] - row['ORIGINAL VOUCHER']:
                    row['Match/Not Match'] = 'Match'
                    row['Type'] = ''
                else:
                    row['Match/Not Match'] = 'Not Match'
                    row['Type'] = 'Count'
            else:
                if row['QC ABUNDANCE'] == row['ORIGINAL ABUNDANCE']:
                    row['Match/Not Match'] = 'Match'
                    row['Type'] = ''
                else:
                    row['Match/Not Match'] = 'Not Match'
                    row['Type'] = 'Count'


    return pd.Series([row['Match/Not Match'], row['Type']])

Now we actually apply the function to each row to build the discrepancy report.

In [169]:
python_discrepancy_report[['Match/Not Match', 'Type']] = python_discrepancy_report.apply(discrepancy_check, axis=1)

This next cell is purely for cosmetic purposes. It also sorts the original report in such a way that is easy to compare with the new report.

In [170]:
sorting_cols = ['SITE', 'Match/Not Match', 'ORIGINAL SPECIES', 'QC SPECIES']
python_discrepancy_report.sort_values(sorting_cols, ascending = [True, False, True, True], inplace = True)
python_discrepancy_report.reset_index(inplace=True)
python_discrepancy_report.drop("index", axis=1, inplace=True)

sorting_cols = ['SITE', 'Match /           Not Match', 'ORIGINAL SPECIES', 'QC SPECIES']
sorted_original_report = original_report.sort_values(sorting_cols, ascending = [True, False, True, True])
sorted_original_report.reset_index(inplace=True)
sorted_original_report.drop("index", axis=1, inplace=True)

python_discrepancy_report.replace(np.nan, '', inplace = True)
sorted_original_report.replace(np.nan, '', inplace = True)
original_report.replace(np.nan, '', inplace = True)
initial.replace(np.nan, '', inplace = True)
qc.replace(np.nan, '', inplace = True)

And here are the dataframes, the initial, the QC data, and the output. I have posted them in the repository as csv files so that you can click on them and view the full thing. Here, only the first 10 lines are displayed.

In [171]:
initial.head(10)

,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,SITE
0,Theora lubrica,7,,B13-8328
1,Psammotreta obesa,1,,B13-8328
2,Leukoma staminea,5,,B13-8328
3,Chione californiensis,2,,B13-8328
4,Caecum californicum,1,,B13-8328
5,Bulla gouldiana,1,,B13-8328
6,Acteocina carinata,4,,B13-8328
7,Nicolea sp A,1,1,B13-8328
8,Diplocirrus sp SD1,1,,B13-8328
9,Tubulanus sp SD1,1,1,B13-8328


In [172]:
qc.head(10)

,QC SPECIES,QC ABUNDANCE,SITE
0,Protomima imitatrix,42,B13-8328
1,Scolelepis (Parascolelepis) texana,1,B13-8328
2,Rudilemboides stenopropodus,18,B13-8328
3,Psammotreta obesa,1,B13-8328
4,Pseudopolydora paucibranchiata,11,B13-8328
5,Prionospio (Prionospio) heterobranchia,4,B13-8328
6,Polycirrus sp,1,B13-8328
7,Prionospio (Prionospio) sp,2,B13-8328
8,Sphaerosyllis californiensis,2,B13-8328
9,Scoletoma sp B,1,B13-8328


In [173]:
python_discrepancy_report.head(10)

,SITE,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,QC SPECIES,QC ABUNDANCE,Match/Not Match,Type
0,B13-8328,ARTHROPODA,2,,,,Not Match,ID
1,B13-8328,Acteocina carinata,4,,,,Not Match,ID
2,B13-8328,Acuminodeutopus heteruropus,4,,,,Not Match,ID
3,B13-8328,Ampelisca brachycladus,1,,,,Not Match,ID
4,B13-8328,Ampharete labrops,2,,,,Not Match,ID
5,B13-8328,Amphideutopus oculatus,101,,Amphideutopus oculatus,105,Not Match,Count
6,B13-8328,Amphiodia digitata,2,,Amphiodia digitata,1,Not Match,Count
7,B13-8328,Apionsoma misakianum,1,,Apionsoma misakianum,2,Not Match,Count
8,B13-8328,Apoprionospio pygmaea,1,,,,Not Match,ID
9,B13-8328,Argopecten ventricosus,1,,,,Not Match,ID


Now below we have the heads of the original report. One is the head of the TRUE original, and the other is the head of the original after being sorted.

In [174]:
sorted_original_report.head(10)

,SITE,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,Unnamed: 4,QC SPECIES,QC ABUNDANCE,Match / Not Match,Type,Lines Involved in Resolve,...,Resolve code,Taxa change (Add / Remove),Abund changes (+/-),Unnamed: 14,Completed by QC officer,Unnamed: 16,Unnamed: 17,RESOLVED SPECIES,RESOLVED ABUNDANCE,Resolution Comments
0,B13-8328,Acteocina carinata,4,,,,,Not Match,ID,,...,,,,,,,,,,
1,B13-8328,Acuminodeutopus heteruropus,4,,,,,Not Match,ID,,...,,,,,,,,,,
2,B13-8328,Ampelisca brachycladus,1,,,,,Not Match,ID,,...,,,,,,,,,,
3,B13-8328,Ampharete labrops,2,,,,,Not Match,ID,,...,,,,,,,,,,
4,B13-8328,Amphideutopus oculatus,101,,,Amphideutopus oculatus,105,Not Match,Count,,...,,,,,,,,,,
5,B13-8328,Amphiodia digitata,2,,,Amphiodia digitata,1,Not Match,Count,,...,,,,,,,,,,
6,B13-8328,Apionsoma misakianum,1,,,Apionsoma misakianum,2,Not Match,Count,,...,,,,,,,,,,
7,B13-8328,Apoprionospio pygmaea,1,,,,,Not Match,ID,,...,,,,,,,,,,
8,B13-8328,Argopecten ventricosus,1,,,,,Not Match,ID,,...,,,,,,,,,,
9,B13-8328,Armandia brevis,22,,,Armandia brevis,21,Not Match,Count,,...,,,,,,,,,,


In [175]:
original_report.head(10)

,SITE,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,Unnamed: 4,QC SPECIES,QC ABUNDANCE,Match / Not Match,Type,Lines Involved in Resolve,...,Resolve code,Taxa change (Add / Remove),Abund changes (+/-),Unnamed: 14,Completed by QC officer,Unnamed: 16,Unnamed: 17,RESOLVED SPECIES,RESOLVED ABUNDANCE,Resolution Comments
0,,,,,,,,,,,...,,,,,Taxa Changed (Note with X),# INDs mis-ID'd (counts change),# INDs mis-counted,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,B13-8328,Oligochaeta,7,,,,,Not Match,ID,,...,,,,,,,,,,
3,B13-8328,,,,,Cossura sp,1,Not Match,ID,,...,,,,,,,,,,
4,B13-8328,Cossura sp A,1,,,,,Not Match,ID,,...,,,,,,,,,,
5,B13-8328,Armandia brevis,22,,,Armandia brevis,21,Not Match,Count,,...,,,,,,,,,,
6,B13-8328,Leitoscoloplos pugettensis,1,,,,,Not Match,ID,,...,,,,,,,,,,
7,B13-8328,,,,,Leitoscoloplos sp A,1,Not Match,ID,,...,,,,,,,,,,
8,B13-8328,Glycera americana,1,,,,,Not Match,ID,,...,,,,,,,,,,
9,B13-8328,,,,,Glycera robusta,1,Not Match,ID,,...,,,,,,,,,,


Now here, I will have the code check the two discrepancy reports for differences.


Now I'll zip up the columns that we are comparing into a list of tuples, then make them into python sets, and compare the two sets to check for differences.

In [176]:
rows = zip(python_discrepancy_report['SITE'], 
           python_discrepancy_report['ORIGINAL SPECIES'], 
           python_discrepancy_report['ORIGINAL VOUCHER'], 
           python_discrepancy_report['QC SPECIES'], 
           python_discrepancy_report['QC ABUNDANCE'], 
           python_discrepancy_report['Match/Not Match'], 
           python_discrepancy_report['Type'])

original_rows = zip(sorted_original_report['SITE'], 
                    sorted_original_report['ORIGINAL SPECIES'], 
                    sorted_original_report['ORIGINAL VOUCHER'], 
                    sorted_original_report['QC SPECIES'], 
                    sorted_original_report['QC ABUNDANCE'], 
                    sorted_original_report['Match /           Not Match'], 
                    sorted_original_report['Type'])

rows = set(rows)
original_rows = set(original_rows)

In [177]:
print "Rows that were in the original that are not in the python generated report:"
for row in original_rows - rows:
    print '   SITE:               %s' % row[0]
    print '   Original Species:   %s' % row[1]
    print '   Original Voucher:   %s' % row[2]
    print '   QC Species:         %s' % row[3]
    print '   QC Abundance:       %s' % row[4]
    print '   Match or No Match:  %s' % row[5]
    print '   Type:               %s' % row[6]
    print '\n'

Rows that were in the original that are not in the python generated report:
   SITE:               
   Original Species:   
   Original Voucher:   
   QC Species:         
   QC Abundance:       
   Match or No Match:  
   Type:               




In [178]:
print "Rows that are in the python report that are not in the original:"
for row in rows - original_rows:
    print '   SITE:               %s' % row[0]
    print '   Original Species:   %s' % row[1]
    print '   Original Voucher:   %s' % row[2]
    print '   QC Species:         %s' % row[3]
    print '   QC Abundance:       %s' % row[4]
    print '   Match or No Match:  %s' % row[5]
    print '   Type:               %s' % row[6]
    print '\n'

Rows that are in the python report that are not in the original:
   SITE:               B13-9173
   Original Species:   ARTHROPODA
   Original Voucher:   
   QC Species:         
   QC Abundance:       
   Match or No Match:  Not Match
   Type:               ID


   SITE:               B13-8328
   Original Species:   ARTHROPODA
   Original Voucher:   
   QC Species:         
   QC Abundance:       
   Match or No Match:  Not Match
   Type:               ID


   SITE:               B13-9481
   Original Species:   Heteronemertea
   Original Voucher:   
   QC Species:         
   QC Abundance:       
   Match or No Match:  Not Match
   Type:               ID


   SITE:               B13-8328
   Original Species:   Facelinidae
   Original Voucher:   
   QC Species:         
   QC Abundance:       
   Match or No Match:  Not Match
   Type:               ID


   SITE:               B13-8328
   Original Species:   Veneridae
   Original Voucher:   
   QC Species:         
   QC Abundance:     

So it seems as if there are certain species getting dropped in the original report. Maybe because they are not useful? Maybe there is something that Chase did in his tool that I am not aware of? Because when I look in the initial data, I can see those species listed above. However, I do not see them in the original discrepancy report.

In [179]:
extra_species = ['ARTHROPODA', 'Heteronemertea', 'Facelinidae', 'Veneridae']

Here's the proof that those species are in the original data

In [180]:
initial[initial['ORIGINAL SPECIES'].isin(extra_species)]

,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,SITE
39,Veneridae,2,,B13-8328
58,Facelinidae,1,,B13-8328
88,ARTHROPODA,2,,B13-8328
171,ARTHROPODA,1,,B13-9173
287,Heteronemertea,1,,B13-9481


And here they are in the python generated report.

In [181]:
python_discrepancy_report[python_discrepancy_report['ORIGINAL SPECIES'].isin(extra_species)]

,SITE,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,QC SPECIES,QC ABUNDANCE,Match/Not Match,Type
0,B13-8328,ARTHROPODA,2,,,,Not Match,ID
29,B13-8328,Facelinidae,1,,,,Not Match,ID
65,B13-8328,Veneridae,2,,,,Not Match,ID
122,B13-9173,ARTHROPODA,1,,,,Not Match,ID
300,B13-9481,Heteronemertea,1,,,,Not Match,ID


But you will see below, that even though those species are listed in the initial data, they do not show up in the original discrepancy report.

In [182]:
original_report[original_report['ORIGINAL SPECIES'].isin(extra_species)]

,SITE,ORIGINAL SPECIES,ORIGINAL ABUNDANCE,ORIGINAL VOUCHER,Unnamed: 4,QC SPECIES,QC ABUNDANCE,Match / Not Match,Type,Lines Involved in Resolve,...,Resolve code,Taxa change (Add / Remove),Abund changes (+/-),Unnamed: 14,Completed by QC officer,Unnamed: 16,Unnamed: 17,RESOLVED SPECIES,RESOLVED ABUNDANCE,Resolution Comments
